Claribel Gonell

EN 705.603.82 - Module 6 - Assignment 6


# Directions

From this load you can answer the following questions using cypher language in the neo4j browser window
Once loaded answer the following questions: Submit the answers in a your notebook.
must show your cypher call(s) for each one. Each may require more than one cypher call.
1) Provide the schema
     db.schema.visualization())

CALL db.schema.visualization

2) Determine the ratio of Female to Males
3) Determining the % of dates that were matches (both selected eachother) 
   Select columns that would prove useful in predicting a match.
   Explain your reasoning.

Be careful with the logic - many participants go on multiple dates - you must figure out the correct way to uniquely identify individuals.  Enter these answers in the notebook with the query.

Now that you have the basics.  Go back to the Speed Dating Data Key.doc and select additional columns that you think would be useful in predicting a match.  Then setup supervised learning to create a model that predicts a match.  There isn't a lot of data so see what performance you can achieve.  You are free to use any model you like.

**Import main neo class**

In [1]:
from py2neo import Graph

**Connect to Neo4J Database**

In [2]:
graph = Graph(password = "Utahlove4!")

**1) Provide schema**

In [3]:
schema = "CALL db.schema.visualization"
graph.run(schema)

nodes,relationships
"[(_-22:Person {constraints: [], indexes: [], name: 'Person'})]",[(_-22)-[:Date {}]->(_-22)]


**2) Determine the ratio of Female to Males**

In [4]:
female = "MATCH (female {gender:0})RETURN count(*) as femt"
results = graph.run(female)
for index, rec in enumerate(results):
    f = rec['femt']

print(f)


268


In [5]:
male = "MATCH (male {gender:1}) RETURN count(*) as mal"
results = graph.run(male)
for index, rec in enumerate(results):
    m = rec['mal']
print(m)

274


In [6]:
#Simple Ratio
ratio = f / (m + f)
print(ratio)

0.4944649446494465


**Determine the % of dates that were matches (both selected each other). Select columns that would prove useful in predicting a match. Explain your reasoning.**

Columnn Selection: race & age were the only attributes that seemed appropriate. 

In [7]:
selected = "MATCH  p = () -[r {match:1}]- ()  RETURN count(p)"
graph.run(selected)

count(p)
2684


In [8]:
all_dates = "MATCH q =()-->() RETURN count(q)"
graph.run(all_dates)

count(q)
8148


In [9]:
characteristics = "MATCH  p = () -[r {match:1}]- ()  RETURN p"
graph.run(characteristics)

p
"(_0)<-[:Date {age_diff: 1, int_corr: 0.16, match: 1, race_diff: 1}]-(_12)"
"(_0)<-[:Date {age_diff: 7, int_corr: 0.28, match: 1, race_diff: 0}]-(_18)"
"(_0)<-[:Date {age_diff: 2, int_corr: 0.61, match: 1, race_diff: 0}]-(_13)"


In [10]:
#percentage of matches
percent = (2684/8148)*100
print(percent)

32.94059891998036


**Go back to the Speed Dating Data Key.doc and select additional columns that you think would be useful in predicting a match.  Then setup supervised learning to create a model that predicts a match.  There isn't a lot of data so see what performance you can achieve.  You are free to use any model you like.**

In [11]:
import warnings
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KernelDensity
from sklearn.svm import SVC


In [12]:
dataframe = pd.read_csv("SpeedDatingData_tiny2.csv")
df = dataframe.copy()
df['age'] = df['age'].fillna(df['age'].mean())
df['age_o'] = df['age_o'].fillna(df['age_o'].mean())
df['race'] = df['race'].fillna(df['race'].mean())
df['race_o'] = df['race_o'].fillna(df['race_o'].mean())
missing = df.isnull().sum()


In [13]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]


In [14]:
# Implementing cross validation
# Instantiating the K-Fold cross validation object with 5 folds
k_folds = KFold(n_splits=5, shuffle=True)
# Iterating through each of the folds in K-Fold
for train_index, val_index in k_folds.split(X):
    print("Train :", train_index, "Test: ", val_index)
    X_train, X_val = X.iloc[train_index, :], X.iloc[val_index, :]
    y_train, y_val = y.iloc[train_index], y.iloc[val_index]

    # Instantiating a RandomForestClassifier model
    rfc_model = RandomForestClassifier()
    svc_model = SVC(kernel='linear')  # Support Vector Machine
    gnb_model = GaussianNB()  # Gaussian

    # Fitting the X_train and y_train datasets to models below
    rfc_model.fit(X_train, y_train)
    svc_model.fit(X_train, y_train)
    gnb_model.fit(X_train, y_train)

    # predictions for vaildation sets
    val_preds_rfc = rfc_model.predict(X_val)

    val_preds_svc = svc_model.predict(X_val)
    val_preds_gnb = gnb_model.predict(X_val)

    # Validation Metrics 
    val_accuracy_rfc = accuracy_score(y_val, val_preds_rfc)
    val_confusion_matrix_rfc = confusion_matrix(y_val, val_preds_rfc)

    val_accuracy_svc = accuracy_score(y_val, val_preds_svc)
    val_confusion_matrix_svc = confusion_matrix(y_val, val_preds_svc)

    val_accuracy_gnb = accuracy_score(y_val, val_preds_gnb)
    val_confusion_matrix_gnb = confusion_matrix(y_val, val_preds_gnb)

    # Printing out the validation metrics
    print(f'Accuracy Score Random Forest: {val_accuracy_rfc}')
    print(f'Accuracy Score SVC: {val_accuracy_svc}')
    print(f'Accuracy Score GaussianNB: {val_accuracy_gnb}')

Train : [   2    3    4 ... 8372 8373 8376] Test:  [   0    1    5 ... 8374 8375 8377]
Accuracy Score Random Forest: 0.7941527446300716
Accuracy Score SVC: 0.8281622911694511
Accuracy Score GaussianNB: 0.8281622911694511
Train : [   0    1    2 ... 8375 8376 8377] Test:  [   8   10   20 ... 8366 8370 8373]
Accuracy Score Random Forest: 0.798926014319809
Accuracy Score SVC: 0.8281622911694511
Accuracy Score GaussianNB: 0.8281622911694511
Train : [   0    1    3 ... 8375 8376 8377] Test:  [   2    6    9 ... 8343 8368 8369]
Accuracy Score Random Forest: 0.8323389021479713
Accuracy Score SVC: 0.8532219570405728
Accuracy Score GaussianNB: 0.8532219570405728
Train : [   0    1    2 ... 8375 8376 8377] Test:  [   3    7   12 ... 8363 8367 8372]
Accuracy Score Random Forest: 0.8059701492537313
Accuracy Score SVC: 0.8322388059701492
Accuracy Score GaussianNB: 0.8322388059701492
Train : [   0    1    2 ... 8374 8375 8377] Test:  [   4   13   15 ... 8357 8359 8376]
Accuracy Score Random Forest: 